<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/SECOP_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# SECOP II

# 1. Instalaciones
# Solo ejecutar si no tienes las librerías instaladas
!pip install pandas requests


import pandas as pd
import requests
from datetime import datetime
import time

def buscar_contratos(limite=100, **filtros):
    """
    Busca contratos en SECOP II con filtros personalizables
    """
    url = "https://www.datos.gov.co/resource/rpmr-utcd.json"

    # Construir parámetros de consulta
    params = {"$limit": limite}

    # Mapeo de nombres de parámetros
    mapeo_campos = {
        'nombre_entidad': 'nombre_de_la_entidad',
        'nit_entidad': 'nit_de_la_entidad',
        'departamento': 'departamento_entidad',
        'municipio': 'municipio_entidad',
        'estado_proceso': 'estado_del_proceso',
        'modalidad': 'modalidad_de_contratacion',
        'tipo_contrato': 'tipo_de_contrato',
        'nombre_contratista': 'nombre_del_contratista',
        'nit_contratista': 'documento_contratista',
        'numero_contrato': 'numero_del_contrato',
        'numero_proceso': 'numero_del_proceso',
        'texto_objeto': 'objeto_del_contrato'
    }

    # Aplicar filtros simples
    for filtro_key, filtro_value in filtros.items():
        if filtro_value and filtro_key in mapeo_campos:
            campo_api = mapeo_campos[filtro_key]
            if filtro_key == 'texto_objeto':
                params[campo_api] = {'$like': f'%{filtro_value}%'}
            else:
                params[campo_api] = filtro_value

    # Construir filtros WHERE para rangos
    where_conditions = []

    if filtros.get('valor_minimo', 0) > 0:
        where_conditions.append(f"valor_del_contrato >= {filtros['valor_minimo']}")

    if filtros.get('valor_maximo', 0) > 0:
        where_conditions.append(f"valor_del_contrato <= {filtros['valor_maximo']}")

    if filtros.get('fecha_firma_desde'):
        where_conditions.append(f"fecha_de_firma >= '{filtros['fecha_firma_desde']}'")

    if filtros.get('fecha_firma_hasta'):
        where_conditions.append(f"fecha_de_firma <= '{filtros['fecha_firma_hasta']}'")

    if filtros.get('fecha_inicio_desde'):
        where_conditions.append(f"fecha_de_inicio_del_contrato >= '{filtros['fecha_inicio_desde']}'")

    if filtros.get('fecha_inicio_hasta'):
        where_conditions.append(f"fecha_de_inicio_del_contrato <= '{filtros['fecha_inicio_hasta']}'")

    if where_conditions:
        params['$where'] = ' AND '.join(where_conditions)

    # Realizar petición con reintentos
    max_intentos = 3
    for intento in range(max_intentos):
        try:
            response = requests.get(url, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()
                return pd.DataFrame(data) if data else pd.DataFrame()

            elif response.status_code == 429:  # Rate limit
                if intento < max_intentos - 1:
                    time.sleep(2 ** intento)
                    continue

            print(f"Error HTTP: {response.status_code}")
            return pd.DataFrame()

        except Exception as e:
            if intento < max_intentos - 1:
                time.sleep(2)
                continue
            print(f"Error: {e}")
            return pd.DataFrame()

    return pd.DataFrame()

def mostrar_columnas_disponibles():
    """Muestra las columnas disponibles en SECOP II"""
    df_muestra = buscar_contratos(limite=1)
    if not df_muestra.empty:
        print("Columnas disponibles en SECOP II:")
        for i, col in enumerate(df_muestra.columns, 1):
            print(f"{i:2d}. {col}")
    return df_muestra.columns.tolist() if not df_muestra.empty else []

def exportar_excel(df, prefijo="secop"):
    """Exporta DataFrame a Excel con timestamp"""
    if df.empty:
        print("Sin datos para exportar")
        return None

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo = f"{prefijo}_{timestamp}.xlsx"

    try:
        df.to_excel(archivo, index=False)
        return archivo
    except Exception as e:
        print(f"Error al exportar: {e}")
        return None

def resumen_busqueda(df):
    """Muestra resumen de resultados"""
    if df.empty:
        print("No se encontraron contratos")
        return

    print(f"Encontrados: {len(df)} contratos")

    # Mostrar primeras 10 columnas
    cols_mostrar = list(df.columns)[:10]
    if len(df.columns) > 10:
        cols_mostrar.append(f"... y {len(df.columns) - 10} más")
    print(f"Columnas: {cols_mostrar}")

    # Verificar objeto del contrato
    if 'objeto_del_contrato' in df.columns:
        objetos_validos = df['objeto_del_contrato'].notna().sum()
        objetos_definidos = df[df['objeto_del_contrato'] != 'NO DEFINIDO']['objeto_del_contrato'].notna().sum()
        print(f"Contratos con objeto definido: {objetos_definidos}/{len(df)} ({objetos_validos} válidos)")

# ==============================================================================
# BLOQUE DE EJECUCIÓN - MODIFICA AQUÍ LOS FILTROS
# ==============================================================================

# Variables de filtros
limite = 10
nombre_entidad = ""
nit_entidad = ""
departamento = ""
municipio = "Bogotá"
estado_proceso = ""
modalidad = ""
tipo_contrato = ""
nombre_contratista = ""
nit_contratista = ""
numero_contrato = ""
numero_proceso = ""
valor_minimo = 0
valor_maximo = 0
fecha_firma_desde = ""
fecha_firma_hasta = ""
fecha_inicio_desde = ""
fecha_inicio_hasta = ""
texto_objeto = ""

# Ejecutar búsqueda
df = buscar_contratos(
    limite=limite,
    nombre_entidad=nombre_entidad,
    nit_entidad=nit_entidad,
    departamento=departamento,
    municipio=municipio,
    estado_proceso=estado_proceso,
    modalidad=modalidad,
    tipo_contrato=tipo_contrato,
    nombre_contratista=nombre_contratista,
    nit_contratista=nit_contratista,
    numero_contrato=numero_contrato,
    numero_proceso=numero_proceso,
    valor_minimo=valor_minimo,
    valor_maximo=valor_maximo,
    fecha_firma_desde=fecha_firma_desde,
    fecha_firma_hasta=fecha_firma_hasta,
    fecha_inicio_desde=fecha_inicio_desde,
    fecha_inicio_hasta=fecha_inicio_hasta,
    texto_objeto=texto_objeto
)

# Guardar resultados
if not df.empty:
    archivo = exportar_excel(df)
    if archivo:
        print(f"Guardado en: {archivo}")
    resumen_busqueda(df)
else:
    print("No se encontraron contratos")

df

Guardado en: secop_20250722_130700.xlsx
Encontrados: 10 contratos
Columnas: ['nivel_entidad', 'codigo_entidad_en_secop', 'nombre_de_la_entidad', 'nit_de_la_entidad', 'departamento_entidad', 'municipio_entidad', 'estado_del_proceso', 'modalidad_de_contrataci_n', 'objeto_a_contratar', 'objeto_del_proceso', '... y 12 más']


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,...,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,Nacional,700613045,MINISTERIO DEL DEPORTE,899999306,Distrito Capital de Bogotá,Bogotá,Modificado,Contratación directa,AUNAR ESFUERZOS TECNICOS ADMINISTRATIVOS Y FIN...,AUNAR ESFUERZOS TECNICOS ADMINISTRATIVOS Y FIN...,...,2024-12-27T00:00:00.000,2025-12-31T00:00:00.000,CO1.PCCNTR.7166637,COID-1057-2024,5442011603,ALCALDIA MUNICIPIO DE TIERRALTA,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,800096807
1,Nacional,700094030,MINISTERIO DEL TRABAJO NIVEL CENTRAL*/,830115226,Distrito Capital de Bogotá,Bogotá,En ejecución,Contratación directa,Prestar servicios profesionales para la Subdir...,Prestar servicios profesionales para la Subdir...,...,2025-01-14T00:00:00.000,2025-12-13T00:00:00.000,CO1.PCCNTR.7233490,CPS-MT-030-2025,110000000,Sandra Liliana Becrre Binasco,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1052382317
2,Territorial,701922320,UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PÚ...,900126860,Distrito Capital de Bogotá,Bogotá,Activo,Contratación directa,SSF-044 Prestar servicios de apoyo a la gestió...,SSF-044 Prestar servicios de apoyo a la gestió...,...,NaN,2025-11-09T00:00:00.000,CO1.PCCNTR.8073363,UAESP-377-2025,15600000,Sandra Viviana Ramirez Reyes,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,53159930
3,Nacional,700655012,EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO ADMI...,900505060,Distrito Capital de Bogotá,Bogotá,En ejecución,Contratación régimen especial,Contratar la prestación de servicios profesion...,Contratar la prestación de servicios profesion...,...,2025-07-16T00:00:00.000,2025-12-30T00:00:00.000,CO1.PCCNTR.8083429,235 - 2025,38966667,JUAN FELIPE OCAMPO SANABRIA,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1110559511
4,Nacional,701428641,CONGRESO DE LA REPÚBLICA - HONORABLE SENADO DE...,899999103,Distrito Capital de Bogotá,Bogotá,Modificado,Contratación directa,Prestación de servicios de apoyo a la gestión ...,Prestación de servicios de apoyo a la gestión ...,...,2025-04-03T00:00:00.000,2025-08-19T00:00:00.000,CO1.PCCNTR.7716674,0236-2025,13600000,alvaro celis carrillo,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,13843630
5,Nacional,704155167,ICBF REGIONAL BOGOTA,899999239,Distrito Capital de Bogotá,Bogotá,Modificado,Contratación directa,PRESTAR SERVICIOS PROFESIONALES A LA REGIONAL ...,PRESTAR SERVICIOS PROFESIONALES A LA REGIONAL ...,...,2025-03-12T00:00:00.000,2025-09-30T00:00:00.000,CO1.PCCNTR.7642921,11010432025,25607645,Laura Victoria Duque Canizales,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1010026039
6,Nacional,701585283,JEFATURA SALUD FUERZA AEREA,901440176,Distrito Capital de Bogotá,Bogotá,terminado,Mínima cuantía,SUMINISTRO Y ADAPTACIÓN DE PRÓTESIS AUDITIVAS ...,SUMINISTRO Y ADAPTACIÓN DE PRÓTESIS AUDITIVAS ...,...,2024-05-15T00:00:00.000,2024-11-30T00:00:00.000,CO1.PCCNTR.6297717,065-01-MDN-CGFM-DIGSA-FAC-JEFSA-2024,74960000,STARKEY HEARING TECHNOLOGIES COLOMBIA S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,830067384
7,Nacional,700218035,SIC SUPERINTENDENCIA DE INDUSTRIA Y COMERCIO,800176089,Distrito Capital de Bogotá,Bogotá,Activo,Contratación directa,Prestación de servicios profesionales para que...,Prestación de servicios profesionales para que...,...,NaN,2025-12-19T00:00:00.000,CO1.PCCNTR.8093853,CONTRATO 1505 DE 2025,25666667,LILIANA XIMENA GUISAO SALCEDO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,52522559
8,Nacional,700638059,MINISTERIO DE AGRICULT

In [4]:
#Analizar DataSet
print(df.loc[0, 'objeto_a_contratar'])
print(df.loc[0, 'url_contrato'])

print(df.iloc[0:5][['nombre_de_la_entidad', 'departamento_entidad']])

AUNAR ESFUERZOS TECNICOS ADMINISTRATIVOS Y FINANCIEROS PARA LA EJECUCION DEL PROYECTO DENOMINADO CONSTRUCCIÓN DE PARQUE RECREODEPORTIVO EN EL CORREGIMIENTO DE MANTAGORDAL DEL MUNICIPIODE TIERRALTA DEPARTAMENTO DE CÓRDOBA.
https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.7236029&isFromPublicArea=True&isModal=true&asPopupView=true
                                nombre_de_la_entidad  \
0                             MINISTERIO DEL DEPORTE   
1             MINISTERIO DEL TRABAJO NIVEL CENTRAL*/   
2  UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PÚ...   
3  EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO ADMI...   
4  CONGRESO DE LA REPÚBLICA - HONORABLE SENADO DE...   

         departamento_entidad  
0  Distrito Capital de Bogotá  
1  Distrito Capital de Bogotá  
2  Distrito Capital de Bogotá  
3  Distrito Capital de Bogotá  
4  Distrito Capital de Bogotá  
